### Query Routing (Route the query to appropriate data source)

### Logical Routing 

> We give the LLM knowledge of the various data sources at our disposal and then let the LLM reason which data source to apply based on the user’s query

In [ ]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_ollama import ChatOllama

# Data model Choose documentation
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["python_docs", "js_docs"] = Field(
        ...,
        description="""Given a user question, choose which datasource would be 
            most relevant for answering their question""",
    )

# LLM with function call 
llm = ChatOllama(model="qwen:0.5b", temperature=0)
structured_llm = llm.with_structured_output(RouteQuery)

# Prompt 
system = """You are an expert at routing a user question to the appropriate data 
    source.

Based on the programming language the question is referring to, route it to the 
    relevant data source."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
router = prompt | structured_llm

In [3]:
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = router.invoke({"question": question})

result.datasource

'python_docs'

In [7]:
question = """Why doesn't the following code work:

import { ChatOpenAI } from "@langchain/openai";
import { z } from "zod";

const llm = new ChatOpenAI({model: "gpt-3.5-turbo", temperature: 0})
const structuredLlm = llm.withStructuredOutput(routeQuery, {name: "RouteQuery"})

"""

result = router.invoke({"question": question})

result.datasource

'js_docs'

In [ ]:
### Runnable 
def choose_route(result):
    if "python_docs" in result.datasource.lower():
        ### Logic here 
        return "chain for python_docs"
    else:
        ### Logic here 
        return "chain for js_docs"

full_chain = router | choose_route

In [9]:
full_chain.invoke({"question": question})

'chain for js_docs'

### Semantic Routing

> semantic routing involves embedding various prompts that represent various data sources alongside the user’s query and then performing vector similarity search to retrieve the most similar prompt

In [10]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import chain
from langchain_ollama import ChatOllama, OllamaEmbeddings

# Two prompts
physics_template = """You are a very smart physics professor. You are great at 
    answering questions about physics in a concise and easy-to-understand manner. 
    When you don't know the answer to a question, you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering 
    math questions. You are so good because you are able to break down hard 
    problems into their component parts, answer the component parts, and then 
    put them together to answer the broader question.

Here is a question:
{query}"""


In [11]:
# Embed prompts
embeddings = OllamaEmbeddings(model="qwen:0.5b")
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

In [ ]:
import numpy as np
np.array(prompt_embeddings).shape #Dimensions

(2, 1024)

In [14]:
# Route question to prompt
@chain
def prompt_router(query):
    # Embed question
    query_embedding = embeddings.embed_query(query)
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)
    print(similarity)
    similarity = similarity[0]
    # Pick the prompt most similar to the input question
    most_similar = prompt_templates[similarity.argmax()]
    return PromptTemplate.from_template(most_similar)

semantic_router = (
    prompt_router
    | ChatOllama(model="qwen:0.5b")
    | StrOutputParser()
)

print(semantic_router.invoke("What's a black hole"))

[[0.16956028 0.16201   ]]
A black hole is a region of space where the gravitational pull is so strong that nothing can escape from it. Black holes are incredibly dense and difficult to understand and study. However, there are many theories and models that attempt to explain the behavior of black holes.
